In [1]:
from esmvalcore.experimental import CFG
from esmvalcore.dataset import Dataset, datasets_to_recipe, datasets_from_recipe
import yaml

In [2]:
session = CFG.start_session('test_session')
session

Session({'auxiliary_data_dir': PosixPath('/home/bandela/esmvaltool_extra_data'),
         'check_level': <CheckLevels.DEFAULT: 3>,
         'compress_netcdf': False,
         'config_developer_file': PosixPath('/home/bandela/src/esmvalgroup/esmvalcore/esmvalcore/config-developer.yml'),
         'config_file': PosixPath('/home/bandela/.esmvaltool/config-user.yml'),
         'diagnostics': None,
         'download_dir': PosixPath('/home/bandela/climate_data'),
         'download_latest_datasets': False,
         'drs': {'CMIP3': 'ESGF',
                 'CMIP5': 'ESGF',
                 'CMIP6': 'ESGF',
                 'CORDEX': 'ESGF',
                 'obs4MIPs': 'ESGF'},
         'exit_on_warning': False,
         'extra_facets_dir': (),
         'log_level': 'info',
         'max_datasets': None,
         'max_parallel_tasks': None,
         'max_years': None,
         'offline': False,
         'output_dir': PosixPath('/home/bandela/esmvaltool_output'),
         'output_file_type':

In [3]:
ds = Dataset(short_name='tas', mip='Amon', project='CMIP5', dataset='CanESM2', ensemble='r1i1p1', exp='historical', timerange='2000/2002')
ds

Dataset:
{'dataset': 'CanESM2',
 'project': 'CMIP5',
 'mip': 'Amon',
 'short_name': 'tas',
 'ensemble': 'r1i1p1',
 'exp': 'historical',
 'timerange': '2000/2002'}

In [4]:
ds.add_ancillary(short_name='areacella', mip='fx', ensemble='r0i0p0')
ds.ancillaries

[Dataset:
 {'dataset': 'CanESM2',
  'project': 'CMIP5',
  'mip': 'fx',
  'short_name': 'areacella',
  'ensemble': 'r0i0p0',
  'exp': 'historical',
  'timerange': '2000/2002'}]

In [5]:
ds.session = session
ds.augment_facets()
ds

Dataset:
{'dataset': 'CanESM2',
 'project': 'CMIP5',
 'mip': 'Amon',
 'short_name': 'tas',
 'ensemble': 'r1i1p1',
 'exp': 'historical',
 'frequency': 'mon',
 'institute': ['CCCma'],
 'long_name': 'Near-Surface Air Temperature',
 'modeling_realm': ['atmos'],
 'original_short_name': 'tas',
 'product': ['output1', 'output2'],
 'standard_name': 'air_temperature',
 'timerange': '2000/2002',
 'units': 'K'}
ancillaries:
  {'dataset': 'CanESM2',
   'project': 'CMIP5',
   'mip': 'fx',
   'short_name': 'areacella',
   'ensemble': 'r0i0p0',
   'exp': 'historical',
   'frequency': 'fx',
   'institute': ['CCCma'],
   'long_name': 'Atmosphere Grid-Cell Area',
   'modeling_realm': ['atmos', 'land'],
   'original_short_name': 'areacella',
   'product': ['output1', 'output2'],
   'standard_name': 'cell_area',
   'units': 'm2'}

In [6]:
ds.find_files()
print(ds.files)
for anc in ds.ancillaries:
    print(anc.files)

[PosixPath('/home/bandela/climate_data/cmip5/output1/CCCma/CanESM2/historical/mon/atmos/Amon/r1i1p1/v20120718/tas_Amon_CanESM2_historical_r1i1p1_185001-200512.nc')]
[PosixPath('/home/bandela/climate_data/cmip5/output1/CCCma/CanESM2/historical/fx/atmos/fx/r0i0p0/v20120410/areacella_fx_CanESM2_historical_r0i0p0.nc')]


In [7]:
cube = ds.load()
cube

<iris 'Cube' of air_temperature / (K) (time: 36; latitude: 64; longitude: 128)>

In [8]:
cell_area = cube.cell_measures()[0]
cell_area

<CellMeasure: cell_area / (m2)  <lazy>  shape(64, 128)>

In [9]:
ds['diagnostic'] = 'diagnostic'

tas = ds.copy()
pr = ds.copy(short_name='pr')
print(yaml.safe_dump(datasets_to_recipe([tas, pr]), sort_keys=False))

diagnostics:
  diagnostic:
    variables:
      tas:
        additional_datasets:
        - mip: Amon
          project: CMIP5
          dataset: CanESM2
          ensemble: r1i1p1
          exp: historical
          timerange: 2000/2002
          version: v20120718
          ancillary_variables:
          - short_name: areacella
            mip: fx
            ensemble: r0i0p0
            version: v20120410
      pr:
        additional_datasets:
        - mip: Amon
          project: CMIP5
          dataset: CanESM2
          ensemble: r1i1p1
          exp: historical
          timerange: 2000/2002
          version: v20120718
          ancillary_variables:
          - short_name: areacella
            mip: fx
            ensemble: r0i0p0
            version: v20120410



In [10]:
ds2 = Dataset(short_name='tas', mip='Amon', project='CMIP5', dataset='GFDL-ESM2G', ensemble='r1i1p1', exp='historical', timerange='*')
ds2.session = session
ds2

Dataset:
{'dataset': 'GFDL-ESM2G',
 'project': 'CMIP5',
 'mip': 'Amon',
 'short_name': 'tas',
 'ensemble': 'r1i1p1',
 'exp': 'historical',
 'timerange': '*'}

In [11]:
ds2.files

[PosixPath('/home/bandela/climate_data/cmip5/output1/NOAA-GFDL/GFDL-ESM2G/historical/mon/atmos/Amon/r1i1p1/v20120412/tas_Amon_GFDL-ESM2G_historical_r1i1p1_198101-198512.nc'),
 PosixPath('/home/bandela/climate_data/cmip5/output1/NOAA-GFDL/GFDL-ESM2G/historical/mon/atmos/Amon/r1i1p1/v20120412/tas_Amon_GFDL-ESM2G_historical_r1i1p1_198601-199012.nc'),
 PosixPath('/home/bandela/climate_data/cmip5/output1/NOAA-GFDL/GFDL-ESM2G/historical/mon/atmos/Amon/r1i1p1/v20120412/tas_Amon_GFDL-ESM2G_historical_r1i1p1_199101-199512.nc'),
 PosixPath('/home/bandela/climate_data/cmip5/output1/NOAA-GFDL/GFDL-ESM2G/historical/mon/atmos/Amon/r1i1p1/v20120412/tas_Amon_GFDL-ESM2G_historical_r1i1p1_199601-200012.nc'),
 PosixPath('/home/bandela/climate_data/cmip5/output1/NOAA-GFDL/GFDL-ESM2G/historical/mon/atmos/Amon/r1i1p1/v20120412/tas_Amon_GFDL-ESM2G_historical_r1i1p1_200101-200512.nc')]

In [12]:
ds2

Dataset:
{'dataset': 'GFDL-ESM2G',
 'project': 'CMIP5',
 'mip': 'Amon',
 'short_name': 'tas',
 'ensemble': 'r1i1p1',
 'exp': 'historical',
 'frequency': 'mon',
 'institute': ['NOAA-GFDL'],
 'long_name': 'Near-Surface Air Temperature',
 'modeling_realm': ['atmos'],
 'original_short_name': 'tas',
 'product': ['output1', 'output2'],
 'standard_name': 'air_temperature',
 'timerange': '198101/200512',
 'units': 'K',
 'version': 'v20120412'}

In [13]:
with open("/home/bandela/src/esmvalgroup/esmvaltool/esmvaltool/recipes/examples/recipe_python_new_ancillary.yml") as file:
    datasets = datasets_from_recipe(yaml.safe_load(file), session)
datasets

[Dataset:
 {'diagnostic': 'map',
  'variable_group': 'tas',
  'dataset': 'BCC-ESM1',
  'project': 'CMIP6',
  'mip': 'Amon',
  'short_name': 'tas',
  'alias': 'CMIP6',
  'end_year': 2000,
  'ensemble': 'r1i1p1f1',
  'exp': 'historical',
  'grid': 'gn',
  'preprocessor': 'select_january',
  'recipe_dataset_index': 0,
  'start_year': 2000}
 ancillaries:
   {'diagnostic': 'map',
    'variable_group': 'tas',
    'dataset': 'BCC-ESM1',
    'project': 'CMIP6',
    'mip': 'fx',
    'short_name': 'areacella',
    'end_year': 2000,
    'ensemble': 'r1i1p1f1',
    'exp': '1pctCO2',
    'grid': 'gn',
    'start_year': 2000},
 Dataset:
 {'diagnostic': 'map',
  'variable_group': 'tas',
  'dataset': 'CanESM2',
  'project': 'CMIP5',
  'mip': 'Amon',
  'short_name': 'tas',
  'alias': 'CMIP5',
  'end_year': 2000,
  'ensemble': 'r1i1p1',
  'exp': 'historical',
  'preprocessor': 'select_january',
  'recipe_dataset_index': 1,
  'start_year': 2000}
 ancillaries:
   {'diagnostic': 'map',
    'variable_group'